In [5]:
import sys
sys.path.insert(0, "../")

### Import General Python libs and modules

In [19]:
from datetime import datetime
from pathlib import Path
import pandas as pd
from pprint import pprint

### Import Feast related modules and definitions from feast_repo directories

In [20]:
from feast import FeatureStore
from entities.entity import zipcode, dob_ssn
from features.feature_views import zipcode_features, credit_history
from feature_service.feature_svc import zipcode_features_svc
from queries.data_fetcher import DataFetcher

In [21]:
# Change this location to yours
FEAST_REPO = "/Users/jules/git-repos/feast_workshops/module_3/feature_repo/"

In [10]:
repo_path = Path(FEAST_REPO)
fs = FeatureStore(repo_path=repo_path)

### Step 1. Register the data source, entity, features and feature view, and feature service in feast_repo with the Feast Registry

In [11]:
fs.apply([zipcode, dob_ssn, zipcode_features, credit_history, zipcode_features_svc])

### Step 2. Now materialize, load data into online store

In [12]:
fs.materialize_incremental(end_date=datetime.utcnow())

Materializing 2 feature views to 2021-09-08 01:21:45-07:00 into the sqlite online store.

zipcode_features from 2011-09-11 01:21:45-07:00 to 2021-09-08 01:21:45-07:00:


100%|███████████████████████████████████████████████████████| 28844/28844 [00:03<00:00, 7410.92it/s]


credit_history from 2021-06-10 01:21:51-07:00 to 2021-09-07 18:21:45-07:00:


100%|███████████████████████████████████████████████████████| 28633/28633 [00:05<00:00, 5170.11it/s]


### Step 3. Fetch training data augmented with loans data

In [25]:
fetcher = DataFetcher(fs, repo_path)
training_df = fetcher.get_training_data()
pd.set_option('display.max_columns', 50)
training_df.head(5)

,event_timestamp,loan_id,dob_ssn,zipcode,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_amnt,loan_int_rate,loan_status,created_timestamp,city,state,location_type,tax_returns_filed,population,total_wages,credit_card_due,mortgage_due,student_loan_due,vehicle_loan_due,hard_pulls,missed_payments_2y,missed_payments_1y,missed_payments_6m,bankruptcies
0,2020-08-25 20:34:41.361000+00:00,38637,19960703_3449,69033,66,42000,RENT,2.0,MEDICAL,6475,9.99,0,2020-08-25 20:34:41.361000+00:00,IMPERIAL,NE,PRIMARY,1171,2141,32596076,1928,1197324,24208,4691,1,4,0,1,0
1,2020-08-25 20:53:02.594000+00:00,38636,19901017_6108,92014,56,150000,MORTGAGE,5.0,PERSONAL,15000,11.48,0,2020-08-25 20:53:02.594000+00:00,DEL MAR,CA,PRIMARY,7681,12635,653566215,1257,152136,42087,26234,1,1,0,0,0
2,2020-08-25 21:11:23.826000+00:00,38635,19880422_2592,68826,65,76000,RENT,3.0,HOMEIMPROVEMENT,35000,10.99,1,2020-08-25 21:11:23.826000+00:00,CENTRAL CITY,NE,PRIMARY,1746,3147,47886607,4089,1384134,31368,28279,3,7,0,0,0
3,2020-08-25 21:29:45.059000+00:00,38634,19681208_6537,24872,54,120000,MORTGAGE,4.0,PERSONAL,17625,7.49,0,2020-08-25 21:29:45.059000+00:00,PANTHER,WV,PRIMARY,252,518,7205163,7580,1489350,36038,6894,0,1,0,0,0
4,2020-08-25 21:48:06.292000+00:00,38633,19491126_1487,43205,57,53000,MORTGAGE,1.0,PERSONAL,5800,13.16,0,2020-08-25 21:48:06.292000+00:00,COLUMBUS,OH,PRIMARY,4826,7185,112085573,388,438726,41429,12736,6,4,0,1,0


### Step 4. Get some inference data for zipcode and dob_ssn

In [17]:
zipcodes_dob_ssns = [(30721, "19790429_9552"), (48893, "19971025_8002")]
for zipcode, dob_ssn in zipcodes_dob_ssns:
    print(f"Fetching feature vector for zipcode and dob_ssn:{zipcode}, {dob_ssn}")
    data = fetcher.get_online_data(zipcode, dob_ssn)
    pprint(data)
    print("------" * 5)

Fetching feature vector for zipcode and dob_ssn:30721, 19790429_9552
{'bankruptcies': [0],
 'city': ['DALTON'],
 'credit_card_due': [4906],
 'dob_ssn': ['19790429_9552'],
 'hard_pulls': [4],
 'location_type': ['PRIMARY'],
 'missed_payments_1y': [1],
 'missed_payments_2y': [6],
 'missed_payments_6m': [1],
 'mortgage_due': [1727236],
 'population': [38542],
 'state': ['GA'],
 'student_loan_due': [32687],
 'tax_returns_filed': [19583],
 'total_wages': [534687864],
 'vehicle_loan_due': [2999],
 'zipcode': [30721]}
------------------------------
Fetching feature vector for zipcode and dob_ssn:48893, 19971025_8002
{'bankruptcies': [0],
 'city': ['WEIDMAN'],
 'credit_card_due': [2320],
 'dob_ssn': ['19971025_8002'],
 'hard_pulls': [1],
 'location_type': ['PRIMARY'],
 'missed_payments_1y': [0],
 'missed_payments_2y': [4],
 'missed_payments_6m': [0],
 'mortgage_due': [1329636],
 'population': [4421],
 'state': ['MI'],
 'student_loan_due': [8914],
 'tax_returns_filed': [2424],
 'total_wages': [7